In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [68]:
def calculate_moving_averages(df, window_sizes):
    for window_size in window_sizes:
        column_name = f'MA_{window_size}'
        df[column_name] = df['close'].rolling(window=window_size).mean()
    return df

In [69]:
df = pd.read_csv('bnb_historical_data.csv')

In [70]:
window_sizes = [7, 25, 99]
df = calculate_moving_averages(df, window_sizes)

In [71]:
features = df[['MA_7', 'MA_25', 'MA_99', 'open', 'high', 'low', 'close', 'volume']].dropna().values
target = df['close'].values.reshape(-1, 1)

# Normalize features using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)

In [72]:
print(features)

[[245.62857143 245.78       246.01313131 ... 245.5        245.6
   87.649     ]
 [245.6        245.768      246.00606061 ... 245.5        245.6
   55.016     ]
 [245.61428571 245.76       246.         ... 245.5        245.7
   37.494     ]
 ...
 [310.31428571 310.428      309.16767677 ... 310.3        310.5
  255.438     ]
 [310.3        310.424      309.19292929 ... 310.4        310.5
  618.347     ]
 [310.38571429 310.408      309.21919192 ... 310.4        310.6
  528.993     ]]


In [73]:
def create_sequences(data, target, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        label = target[i + sequence_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [74]:
sequence_length = 10  # Adjust as needed
X, y = create_sequences(features_scaled, target, sequence_length)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Adjust dropout rate as needed
model.add(LSTM(units=50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [77]:
model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/1000
11833/11833 [==============================] - 44s 4ms/step - loss: 959.7989 - val_loss: 49.1938
Epoch 2/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 669.5600 - val_loss: 12.7152
Epoch 3/1000
11833/11833 [==============================] - 44s 4ms/step - loss: 643.9407 - val_loss: 2.1913
Epoch 4/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 592.5128 - val_loss: 23.4720
Epoch 5/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 689.9058 - val_loss: 3.0674
Epoch 6/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 561.7343 - val_loss: 60.3394
Epoch 7/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 541.8173 - val_loss: 59.4547
Epoch 8/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 539.0456 - val_loss: 8.3983
Epoch 9/1000
11833/11833 [==============================] - 43s 4ms/step - loss: 487.4941 - val_loss: 24.1570
Epoch 10/1000

In [ ]:
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

3287/3287 [==============================] - 5s 1ms/step - loss: 32.9150
Mean Squared Error on Test Set: 32.914955139160156


In [ ]:
import pickle

In [ ]:
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [ ]:
df_24 = pd.read_csv('bnb_historical_data_24.csv')

In [ ]:
window_sizes = [7, 25, 99]
df_24 = calculate_moving_averages(df_24, window_sizes)

In [ ]:
features_24 = df_24[['MA_7', 'MA_25', 'MA_99', 'open', 'high', 'low', 'close', 'volume']].dropna()
target_24 = features_24['close'].values.reshape(-1, 1)
features_24 = features_24.values

scaler_24 = MinMaxScaler(feature_range=(0, 1))
features_scaled_24 = scaler_24.fit_transform(features_24)

In [ ]:
sequence_length = 10  # Adjust as needed
X_24, y_24 = create_sequences(features_scaled_24, target_24, sequence_length)

In [ ]:
loss_24 = loaded_model.evaluate(X_24, y_24)
print(f'Mean Squared Error on Test Set: {loss_24}')

2591/2591 [==============================] - 4s 1ms/step - loss: 6052.9395
Mean Squared Error on Test Set: 6052.939453125
